Enfoque del dataset de noticias falsas basado en algoritmos clásicos de aprendizaje automático

In [13]:
### IMPORTS ###
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Cargar Dataset

In [14]:
### DRIVE DATA ###
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
PATH = "/content/drive/My Drive/TFM/Data/FakeNews_Multi/"

Mounted at /content/drive


In [15]:
### CONJUNTOS DE ENTRENAMIENTO, VALIDACIÓN Y TEST ###
df = pd.read_csv(PATH + "train.csv",  sep=',', on_bad_lines='skip')
df = df[['Text', 'Labels']]
df=df.rename(columns={"Text": "Text", "Labels": "Label"})

X = df['Text']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=55, stratify=y)

print('Tamaño conjunto de Entrenamiento:', len(X_train))
print('Tamaño conjunto de Test:', len(X_test))

Tamaño conjunto de Entrenamiento: 8192
Tamaño conjunto de Test: 2048


# Label Encoding

No es necesario hacer un encoding ya que viene etiquetado de manera numérica

# Limpieza y Representación de Textos

In [16]:
### LIMPIEZA DE TEXTOS ###
stopwords_en = stopwords.words("english")
def clean_text(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords_en]
    # usar los stems
    tokens = [PorterStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 3 caráceres
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial 
    min_length = 3
    p = re.compile('^[a-zA-Z]+$');
    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)
            
    return filtered_tokens

# Bolsa de Palabras

In [17]:
### BOLSA DE PALABRAS ###
X_train = X_train.tolist()
X_test = X_test.tolist()

# entrenamos un modelo de bolsa de palabras
bow = CountVectorizer(analyzer=clean_text).fit(X_train)
# transformamos el conjunto de entrenamiento a bolsa de palabras
X_train_bow = bow.transform(X_train)
# transformamos el conjunto de evaluación a bolsa de palabras
X_test_bow=bow.transform(X_test)

print("Tamaño del vocabulario: ", len(bow.vocabulary_))

Tamaño del vocabulario:  6764


# TF-IDF

In [18]:
### TF-IDF ###
# entrenamos un modelo tf-idf 
tfidf_transformer = TfidfTransformer().fit(X_train_bow)
# transformamos el conjunto de entrenamiento
X_train_tfidf = tfidf_transformer.transform(X_train_bow)
# transformamos el conjunto de entrenamiento
X_test_tfidf = tfidf_transformer.transform(X_test_bow)

# Clasificación Clásica
Se crea un pipeline que ejecuta una secuencia de procesos:


1.   La representación de los textos en bolsa de palabras (CountVectorizer), que recibe como entrada los textos, y se les aplica dentro del CountVectorizer la función clean_text para limpiarlos y reducir el ruido. 
2.   La representación en tf-idf (TfidfTransformer), recibe como entrada la salida del proceso 1, y produce los vectores tf-idf. 
3. El clasificador SVC, Logistic Regression o Random Forest Clasiffier.

In [19]:
### PIPELINE SVM ###
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),  
    ('tf', TfidfTransformer()),  
    ('svm', SVC()), 
])

# Parámetros para el algoritmo SVM
grid_params_svm = [{'svm__kernel': ['linear', 'rbf'], 
                    'svm__C': [0.1, 1], # [0.1, 1, 10, 100, 1000]
                    'svm__gamma':  [1, 0.1] # [1, 0.1, 0.01, 0.001, 0.0001]
                    }]
gs = GridSearchCV(pipeline, param_grid=grid_params_svm, 
                  scoring='accuracy', cv=5, verbose = 1)

# entrenamos el grid
gs.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs.best_params_)
print('Mejor accuracy: %.3f' % gs.best_score_)
print(gs.best_estimator_)

best_svm = gs.best_estimator_
predictions = best_svm.predict(X_test)
print(classification_report(y_test, predictions))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Los mejores parámetros son : {'svm__C': 1, 'svm__gamma': 1, 'svm__kernel': 'rbf'}
Mejor accuracy: 0.241
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7f8c386a1430>)),
                ('tf', TfidfTransformer()), ('svm', SVC(C=1, gamma=1))])
              precision    recall  f1-score   support

           0       0.24      0.12      0.16       331
           1       0.26      0.40      0.31       399
           2       0.24      0.36      0.29       423
           3       0.23      0.27      0.25       392
           4       1.00      0.02      0.05       168
           5       0.24      0.11      0.15       335

    accuracy                           0.24      2048
   macro avg       0.37      0.21      0.20      2048
weighted avg       0.30      0.24      0.22      2048



In [20]:
### PIPELINE LOGISTIC REGRESSION ###
pipeline2 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),  
    ('tf', TfidfTransformer()),  
    ('lr', LogisticRegression(random_state=0)), 
])

# Parámetros para el algoritmo Logistic Regression
grid_params_lr = [{'lr__penalty': ['l1', 'l2'], 
                    'lr__C': [1.0, 0.5],
                    'lr__solver':  ['liblinear']
                    }]
gs2 = GridSearchCV(pipeline2, param_grid=grid_params_lr, 
                  scoring='accuracy', cv=5, verbose = 1)

# entrenamos el grid
gs2.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs2.best_params_)
print('Mejor accuracy: %.3f' % gs2.best_score_)
print(gs2.best_estimator_)

best_svm = gs2.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Los mejores parámetros son : {'lr__C': 0.5, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
Mejor accuracy: 0.235
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7f8c386a1430>)),
                ('tf', TfidfTransformer()),
                ('lr',
                 LogisticRegression(C=0.5, penalty='l1', random_state=0,
                                    solver='liblinear'))])
              precision    recall  f1-score   support

           0       0.22      0.05      0.08       331
           1       0.22      0.49      0.30       399
           2       0.25      0.31      0.28       423
           3       0.24      0.26      0.25       392
           4       0.20      0.01      0.01       168
           5       0.27      0.11      0.15       335

    accuracy                           0.24      2048
   macro avg       0.23      0.20      0.18      2048
weighted avg       0.24      0

In [21]:
### PIPELINE RANDOM FOREST ###
pipeline3 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),  
    ('tf', TfidfTransformer()),  
    ('rfc', RandomForestClassifier(random_state=0)), 
])

# Parámetros para el algoritmo Random Forest
grid_params_rfc = [{'rfc__criterion': ['gini', 'entropy'], 
                    'rfc__max_depth': [9, 10],
                    'rfc__min_samples_split':  [10]
                    }]
gs3 = GridSearchCV(pipeline3, param_grid=grid_params_rfc, 
                  scoring='accuracy', cv=5, verbose = 1)

# entrenamos el grid
gs3.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs3.best_params_)
print('Mejor accuracy: %.3f' % gs3.best_score_)
print(gs3.best_estimator_)

best_svm = gs3.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Los mejores parámetros son : {'rfc__criterion': 'gini', 'rfc__max_depth': 10, 'rfc__min_samples_split': 10}
Mejor accuracy: 0.221
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7f8c386a1430>)),
                ('tf', TfidfTransformer()),
                ('rfc',
                 RandomForestClassifier(max_depth=10, min_samples_split=10,
                                        random_state=0))])
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       331
           1       0.31      0.18      0.23       399
           2       0.21      0.83      0.34       423
           3       0.27      0.13      0.17       392
           4       0.00      0.00      0.00       168
           5       0.00      0.00      0.00       335

    accuracy                           0.23      2048
   macro avg       0.13      0.19      0.12      2048
weighted a

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
